In [332]:
import numpy as np
from scipy.stats import binom
import pandas as pd
from itertools import combinations, product

carga = 2850

In [333]:
class Agrupamento:
    def __init__(self, pot, qtd, prob):
        self.pot = pot
        self.qtd = qtd
        self.prob = prob
        self.geraTabela()

        self.pot_max = self.pot * self.qtd

    def geraTabela(self):
        self.resultados = []
        self.iterado = []
        for i in range(int(self.qtd) + 1):
            self.resultados.append((i * self.pot, binom.pmf(i, int(self.qtd), self.prob)))
            self.iterado.append(0)
        self.usina = pd.DataFrame(self.resultados,columns=['Pot','Prob'])
        self.usina['Iter'] = self.iterado
        self.usina['IterAg'] = np.nan


In [334]:
def Calcula_COPT(data, carga, falhas_simultaneas):
    agrupamento = []
    agrpot = []
    agrprob = []
    resul = []
    usinas = []
    geracao_total = 0
    for i in range(len(data['Usina'])):
        agrupamento.append(Agrupamento(data.loc[i]['Pot. Ativa Max'], data.loc[i]['Unidades'], data.loc[i]['FOR']))
        agrpot.append(list(zip(*agrupamento[i].resultados))[0])
        agrprob.append(list(zip(*agrupamento[i].resultados))[1])
        usinas.append(agrupamento[i].usina)
        geracao_total += data.loc[i]['Unidades']*data.loc[i]['Pot. Ativa Max']
    
    potencia_para_falta = geracao_total - carga
    
    lista = []
    for i in range(len(usinas)):
        for x in range(len(usinas[i])):
            lista.append((i,x))

    # se 'falhas_simultaneas' for um numero inteiro, usar apenas esse numero de falhas
    if isinstance(falhas_simultaneas, int):
        resultado_pot_teste = []
        resultado_prob_teste = []
        
        n_usinas = len(usinas)
        prod_prob = list(product(*agrprob))

        for it, comb in enumerate(product(*agrpot)):

            # o numero de zeros é o numero de usinas sem falha; 
            # caso haja mais usinas sem falha, armazenar a potencia e a probabilidade
            if comb.count(0) >= n_usinas - falhas_simultaneas:
                resultado_pot_teste.append(sum(comb))
                resultado_prob_teste.append(np.prod(prod_prob[it]))

            # caso haja mais usinas com falha, não armazenar. continua p/ proxima combinação


    # se 'falhas_simultaneas' não for inteiro, fazer com todas as possibilidades
    else:
        resultado_pot_teste = []
        for i in product(*agrpot):
            resultado_pot_teste.append(sum(i))
        resultado_prob_teste = []
        for i in product(*agrprob):
            resultado_prob_teste.append(np.prod(i))



    copt = pd.DataFrame(resultado_pot_teste, columns=['Pot indisponivel'])
    copt['Probabilidade'] = resultado_prob_teste
    copt = copt.groupby(by = ['Pot indisponivel'], as_index = False).sum()
    
    perda_de_carga = []
    
    for i in range(len(copt)):
        if copt.loc[i]['Pot indisponivel'] <=  potencia_para_falta:
            perda_de_carga.append(0)
        else:
            perda_de_carga.append(abs(copt.loc[i]['Pot indisponivel'] - potencia_para_falta))
    
    
    copt['Perda de Carga'] = perda_de_carga
    copt['x.p'] = copt['Perda de Carga'] * copt['Probabilidade']
    
    EPNS = sum(copt['x.p'])
    LOLP = sum(copt[copt['Perda de Carga'] > 0]['Probabilidade'])
    LOLE = LOLP * 8760
    EENS = EPNS * 8760
    resul_copt = [EPNS, EENS, LOLP, LOLE]
    return resul_copt,copt



def pu(x):
    return x/100

In [335]:
# data = pd.read_excel('Gerac.xlsx')
# data['FOR'] = data['FOR'].apply(pu)


# Agrupar usinas que tenham as mesmas potências e FORs

data = pd.read_excel('Gerac.xlsx')

data2 = data.groupby(by = ['Pot. Ativa Max','FOR'], as_index=False, sort=False,group_keys=False)['Unidades'].sum().reset_index()
data2['Usina'] = data['Usina']
data = data2

data['FOR'] = data['FOR'].apply(pu)

# data

In [336]:
dados_carga = pd.read_excel('dataframe_curva_de_carga.xlsx')

# dados_carga

In [337]:
# Número de falhas simultâneas ('all' para utilizar todas as combinações possíveis)
FALHAS_SIMULTANEAS = 'all'

In [338]:
EPNS = []
EENS = []
LOLP = []
LOLE = []
valores_de_carga = dados_carga['Valor de Carga'].values

for n,i in enumerate(valores_de_carga):
    result, copt = Calcula_COPT(data, carga*i/100, FALHAS_SIMULTANEAS)
    EPNS.append(result[0]*dados_carga['Probabilidade'][n])
    EENS.append(result[1]*dados_carga['Probabilidade'][n])
    LOLP.append(result[2]*dados_carga['Probabilidade'][n])
    LOLE.append(result[3]*dados_carga['Probabilidade'][n])
    
LOLP = sum(LOLP)
EPNS = sum(EPNS)
LOLE = sum(LOLE)
EENS = sum(EENS)

In [339]:
E_sob_curva = sum(dados_carga['Valor de Carga'] * dados_carga['Probabilidade'] * 8760) * carga

EIR = 1 - EENS/E_sob_curva

In [340]:
precisao = 8

print("LOLP: ", round(LOLP, precisao),
    "\nEPNS: ", round(EPNS, precisao), "\t MW",
    "\nLOLE: ", round(LOLE, precisao), "\t h/ano",
    "\nEENS: ", round(EENS, precisao), "\t MWh/ano",
    "\nEIR : ", round(EIR , precisao))

LOLP:  0.02338138 
EPNS:  3.63670173 	 MW 
LOLE:  204.82085416 	 h/ano 
EENS:  31857.50716941 	 MWh/ano 
EIR :  0.99998458
